In [1]:
import torch
import os
import sys
from pathlib import Path
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('../utils/')
from configuration import *
from utils import *
import FigureConfig as FC
import pNN_Power_Aware as pNN

args = parser.parse_args([])

In [2]:
result = torch.load('result.matrix')
result.shape

torch.Size([13, 10, 51, 8])

In [3]:
import numpy as np

target = [[ 200.,  240.,  400.,  500.,  300.,  300.,  200.,  300.,  600.,  280.,  340.,  280.,  240.],
          [ 400.,  480.,  800., 1000.,  600.,  600.,  400.,  600., 1200.,  560.,  680.,  560.,  480.],
          [ 600.,  720., 1200., 1500.,  900.,  900.,  600.,  900., 1800.,  840., 1020.,  840.,  720.],
          [ 800.,  960., 1600., 2000., 1200., 1200.,  800., 1200., 2400., 1120., 1360., 1120.,  960.]]
target = np.array(target)

gammas = np.linspace(0, 1, 51)

In [4]:
def calculate_hessian_eigenvalues(model, dataloader, loss_function):
    eigenvalues = {}

    model.train()
    
    for images, labels in dataloader:
        
        # 前向传播
        outputs = model(images)
        loss = loss_function(model, images, labels)
        
        # 计算梯度
        model.zero_grad()
        loss.backward(create_graph=True)
        
        for n, p in model.named_parameters():
            if p.requires_grad:
                grad = p.grad.view(-1)
                hessian = torch.zeros(grad.size(0), grad.size(0))
                # 计算Hessian的每个元素
                for i in range(len(grad)):
                    grad2 = torch.autograd.grad(grad[i], p, retain_graph=True, allow_unused=True)[0].view(-1)
                    if grad2 is not None:
                        hessian[i, :] = grad2
                
                # 计算Hessian的特征值
                eigvals = torch.linalg.eigvals(hessian)
                eigenvalues[n] = eigvals
    
    return eigenvalues

In [5]:
values = []

for ds in range(13):
    values_ds = []
    
    args.DATASET = ds
    train_loader, datainfo = GetDataLoader(args, 'train', path='../dataset/')
    power = result[ds,:,:,7]
    acc = result[ds,:,:,6]
    
    print(datainfo)
    
    for i in range(4):
        values_target = []
        
        t = target[i, ds]
        
        valid = power < t
        valid_acc = acc[valid]
        best_acc = valid_acc.max()
        seed = torch.where(acc == best_acc)[0][0].item()
        gamma_idx = torch.where(acc == best_acc)[1][0].item()
        gamma = gammas[gamma_idx]

        rounded_num = np.round(gamma, 3)
        formatted_num = ('{:.3f}'.format(rounded_num)).rstrip('0')
        if formatted_num[-1] == '.':
            formatted_num += '0'
        if rounded_num == 0 or rounded_num == 1:
            formatted_num = '{:.1f}'.format(rounded_num)
        gamma = formatted_num
        
        setup = f"data_{datainfo['dataname']}_seed_{seed}_Penalty_power_Factor_{gamma}"
        print(setup)

        model_FT_name = f'./models/pNN_{setup}_FT.model'
        model_FT = torch.load(model_FT_name)

        args.powerestimator = 'power'
        args.powerbalance = float(gamma)
        loss_function = pNN.Lossfunction(args).to(args.DEVICE)

        # 计算 Hessian 矩阵的特征值
        eigenvalues = calculate_hessian_eigenvalues(model_FT, train_loader, loss_function)

        # 打印 Hessian 矩阵的特征值
        for n, eigvals in eigenvalues.items():
            # Sort eigenvalues by their absolute values in descending order
            top_eigvals = sorted(eigvals, key=lambda x: abs(x), reverse=True)[:3]
            print(f'Top 3 eigenvalues for Hessian of {n}:')
            for eigval in top_eigvals:
                values_target.append(abs(eigval).item())
                print(abs(eigval))

        values_ds.append(values_target)
    values.append(values_ds)

{'dataname': 'acuteinflammation', 'N_feature': 6, 'N_class': 2, 'N_train': 70, 'N_valid': 23, 'N_test': 25}
data_acuteinflammation_seed_0_Penalty_power_Factor_1.0


/home/kit/tm/px3192/.local/lib/python3.9/site-packages/torch/autograd/graph.py:744: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at ../torch/csrc/autograd/engine.cpp:1203.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Top 3 eigenvalues for Hessian of act.rt_:
tensor(2.0333e-05)
tensor(6.9901e-07)
tensor(3.7682e-07)
Top 3 eigenvalues for Hessian of inv.rt_:
tensor(1.1475e-08)
tensor(2.3595e-09)
tensor(1.9654e-09)
Top 3 eigenvalues for Hessian of model.0-th pLayer.theta_:
tensor(0.0367)
tensor(0.0308)
tensor(0.0285)
Top 3 eigenvalues for Hessian of model.1-th pLayer.theta_:
tensor(1.4901e-08)
tensor(0.)
tensor(0.)
data_acuteinflammation_seed_2_Penalty_power_Factor_0.96
Top 3 eigenvalues for Hessian of act.rt_:
tensor(0.0757)
tensor(1.8460e-05)
tensor(4.1202e-06)
Top 3 eigenvalues for Hessian of inv.rt_:
tensor(1.1054e-07)
tensor(4.9951e-09)
tensor(4.6586e-09)
Top 3 eigenvalues for Hessian of model.0-th pLayer.theta_:
tensor(9.0013)
tensor(4.2206)
tensor(1.4722)
Top 3 eigenvalues for Hessian of model.1-th pLayer.theta_:
tensor(0.3102)
tensor(0.0496)
tensor(0.0128)
data_acuteinflammation_seed_0_Penalty_power_Factor_0.22
Top 3 eigenvalues for Hessian of act.rt_:
tensor(5.5566)
tensor(0.0384)
tensor(0.006

In [6]:
values = torch.tensor(values)
top_values, top_indices = values.topk(6, dim=-1)

# top_values will have the shape [13, 4, 3]
print("Shape of the resulting tensor:", top_values.shape)

Shape of the resulting tensor: torch.Size([13, 4, 6])


In [7]:
torch.save(top_values, '../RESULT/penalty.hessian')

In [8]:
for ds in range(13):
    print(top_values[ds])

tensor([[3.6746e-02, 3.0848e-02, 2.8546e-02, 2.0333e-05, 6.9901e-07, 3.7682e-07],
        [9.0013e+00, 4.2206e+00, 1.4722e+00, 3.1024e-01, 7.5705e-02, 4.9574e-02],
        [6.0807e+03, 5.9008e+02, 5.5566e+00, 4.6159e+00, 8.9861e-01, 3.4689e-01],
        [6.0807e+03, 5.9008e+02, 5.5566e+00, 4.6159e+00, 8.9861e-01, 3.4689e-01]])
tensor([[2.7652e+00, 1.0428e+00, 8.4336e-01, 4.5987e-01, 6.6101e-02, 7.1415e-03],
        [5.0950e+01, 1.4384e+00, 8.6311e-01, 2.4529e-01, 3.4391e-02, 2.7613e-02],
        [3.2208e+02, 1.3369e+01, 1.1517e+01, 5.6225e-01, 1.0921e-01, 9.7534e-02],
        [1.7962e+01, 1.1646e+01, 4.9933e+00, 1.0732e+00, 6.0425e-01, 2.9206e-01]])
tensor([[1.0497e+01, 4.8803e+00, 2.1834e+00, 1.2319e+00, 1.0072e+00, 1.0014e+00],
        [1.8256e+01, 7.5818e+00, 7.2129e+00, 2.5222e+00, 4.3197e-01, 2.2694e-01],
        [1.8256e+01, 7.5818e+00, 7.2129e+00, 2.5222e+00, 4.3197e-01, 2.2694e-01],
        [1.8173e+01, 3.3898e+00, 5.9085e-01, 1.7156e-01, 1.2535e-02, 2.5505e-03]])
tensor([[3.34